# column_transformer 

In [1]:
import pandas as pd 
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

In [3]:
df=pd.read_csv('covid_toy.csv')

In [4]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
92,82,Female,102.0,Strong,Kolkata,No
31,83,Male,103.0,Mild,Kolkata,No
61,81,Female,98.0,Strong,Mumbai,No
72,83,Female,101.0,Mild,Kolkata,No
62,56,Female,104.0,Strong,Bangalore,Yes


In [5]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                               test_size=0.2,
                                              random_state=1)

In [11]:
X_train.head(2)

,age,gender,fever,cough,city
2,42,Male,101.0,Mild,Delhi
73,34,Male,98.0,Strong,Kolkata


In [19]:
# sdding simple imputer to fever col

si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
#also the test data 

X_test_fever=si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [20]:
# ordinalEncoding to cough

Oe=OrdinalEncoder(categories=[['Mild','Strong']])

X_train_cough=Oe.fit_transform(X_train[['cough']])

X_test_cough=Oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [21]:
#Onehot Encoding 
ohe=OneHotEncoder(drop='first',sparse=False)

X_train_gen_city=ohe.fit_transform(X_train[['gender','city']])

X_test_gen_city=ohe.fit_transform(X_test[['gender','city']])

X_train_gen_city.shape

(80, 4)

In [26]:
#extracting Age
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age=X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [29]:
X_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gen_city,X_train_cough),axis=1)

X_test_transformed=np.concatenate((X_test_age,X_test_fever,X_test_gen_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

 # Now Using ColumnTransformer 

In [30]:
from sklearn.compose import ColumnTransformer

In [31]:
transformer =ColumnTransformer(transformers=[
    #always pass transformer in tuples 
    # (Transformer_name)
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [33]:
transformer.fit_transform(X_train).shape

(80, 7)

In [37]:
transformer.transform(X_test).shape

(20, 7)